In [ ]:
# techs used - duckdb for a local dummy database
# columnar database
import duckdb
import timeit
import pandas as pd

In [ ]:
### DDL to instantiate database ###

# create initial database connection (and instantiate database if it doesn't exist already)
con = duckdb.connect('dim-model.db')

# create two separate schemas
con.sql("""
CREATE SCHEMA snow;
CREATE SCHEMA star;
""")

In [ ]:
# create a table using the initial data - auto create columns based on information
sql = """
CREATE TABLE raw_companies AS
SELECT * FROM read_csv_auto('SorensonworkfirmFIVEdata.csv')
"""
con.sql(sql)

In [ ]:
con.table('raw_companies')

┌───────┬──────────────────────┬────────────┬────────┬───────────────┬───┬───────┬─────────┬───────┬────────┬───────┐
│ YEAR  │       FIRMNAME       │ FIVEFIRMID │ FIRMID │ CUSIPHEADERID │ … │ COMM  │ MONITOR │ DISK  │ MEMORY │ BOARD │
│ int64 │       varchar        │   int64    │ int64  │    varchar    │   │ int64 │  int64  │ int64 │ int64  │ int64 │
├───────┼──────────────────────┼────────────┼────────┼───────────────┼───┼───────┼─────────┼───────┼────────┼───────┤
│  1987 │ Accent Systems Corp. │          1 │      1 │ NULL          │ … │     0 │       0 │     0 │      0 │     0 │
│  1982 │ Adage, Inc.          │          2 │      2 │ NULL          │ … │     0 │       0 │     0 │      0 │     0 │
│  1983 │ Adage, Inc.          │          2 │      2 │ NULL          │ … │     0 │       0 │     0 │      0 │     0 │
│  1984 │ Adage, Inc.          │          2 │      2 │ NULL          │ … │     0 │       0 │     0 │      0 │     0 │
│  1985 │ Adage, Inc.          │          2 │      2 │ N

In [ ]:
# verify initial data types
sql = """
DESCRIBE raw_companies;
"""

initial_cols_dict = {col[0]:col[1] for col in con.sql(sql).fetchall()}

In [ ]:
initial_cols_dict

{'YEAR': 'BIGINT',
 'FIRMNAME': 'VARCHAR',
 'FIVEFIRMID': 'BIGINT',
 'FIRMID': 'BIGINT',
 'CUSIPHeaderName': 'VARCHAR',
 'CUSIPHISTORYID': 'VARCHAR',
 'CUSIPHistoryName': 'VARCHAR',
 'FOUND': 'BIGINT',
 'FATE': 'VARCHAR',
 'ESTATE': 'BIGINT',
 'SALES': 'DOUBLE',
 'EMPLOY': 'DOUBLE',
 'ZIP': 'BIGINT',
 'LAT': 'DOUBLE',
 'LON': 'DOUBLE',
 'RANDD': 'BIGINT',
 'PRODUCTS': 'BIGINT',
 'PTYPES': 'BIGINT',
 'SYSTEM': 'BIGINT',
 'GRAPHIC': 'BIGINT',
 'CEO': 'VARCHAR',
 'OWN': 'BIGINT',
 'CPU': 'BIGINT',
 'OS': 'BIGINT',
 'APPS': 'BIGINT',
 'COMM': 'BIGINT',
 'MONITOR': 'BIGINT',
 'DISK': 'BIGINT',
 'MEMORY': 'BIGINT',
 'BOARD': 'BIGINT',
 'CUSIPHEADERID': 'VARCHAR'}

## A snowflake data model

Steps of Kimball dimensional modeling:

1. Select the business process
2. Declare the grain
3. Identify the dimensions
4. Identify the facts

![](snow.png)

There's a step before this:

0. Know thy data

In [ ]:
# check lat/long/zip data columns
con.sql('SELECT DISTINCT lat, lon, zip FROM raw_companies')

┌───────────────────┬───────────────────┬───────┐
│        LAT        │        LON        │  ZIP  │
│      double       │      double       │ int64 │
├───────────────────┼───────────────────┼───────┤
│ 0.705875992774963 │ -1.39546799659729 │ 15213 │
│ 0.742671012878418 │ -1.24357903003693 │  1821 │
│  0.74406099319458 │ -1.24499404430389 │  1851 │
│ 0.574072003364563 │ -2.04558706283569 │ 92121 │
│ 0.742465019226074 │ -1.24769496917725 │  1460 │
│ 0.743335008621216 │ -1.24683105945587 │  1886 │
│ 0.590389013290405 │ -2.06493091583252 │ 90501 │
│ 0.597449004650116 │ -2.07774996757507 │ 93010 │
│ 0.531078994274139 │ -1.47060298919678 │ 32301 │
│ 0.743398010730743 │  -1.2454240322113 │  1824 │
│         ·         │          ·        │    ·  │
│         ·         │          ·        │    ·  │
│         ·         │          ·        │    ·  │
│ 0.596495985984802 │ -2.07023811340332 │ 91367 │
│ 0.746240019798279 │ -1.24732995033264 │  3060 │
│ 0.748165011405945 │ -1.24813294410706 │  3054 │


In [ ]:
### INITIAL DATA CLEANSING ###

# change column zip to type varchar
sql = """
ALTER TABLE raw_companies ALTER COLUMN zip SET DATA TYPE VARCHAR;
"""

# ensure leading 0's present in zip
sql += """
UPDATE raw_companies
SET zip = RIGHT('000' || zip, 5);
"""

# convert lat/long variables to degrees from radians
sql += f"""
UPDATE raw_companies
SET lat = DEGREES(lat), lon = DEGREES(lon);
"""

con.sql(sql)

In [ ]:
# display the results of cleansing
con.sql('SELECT DISTINCT lat, lon, zip FROM raw_companies')

┌────────────────────┬─────────────────────┬─────────┐
│        LAT         │         LON         │   ZIP   │
│       double       │       double        │ varchar │
├────────────────────┼─────────────────────┼─────────┤
│ 40.443715245612374 │  -79.95442665060105 │ 15213   │
│  42.55191460463936 │  -71.25182991208872 │ 01821   │
│ 42.631554610361704 │  -71.33290425753633 │ 01851   │
│ 32.891902929409454 │ -117.20350532704735 │ 92121   │
│ 42.540112037753566 │  -71.48765585356176 │ 01460   │
│  42.58995875831634 │  -71.43815747264637 │ 01886   │
│ 33.826798732433275 │ -118.31182646328722 │ 90501   │
│  34.23130644074354 │ -119.04630402549516 │ 93010   │
│ 30.428584959960574 │  -84.25934462029849 │ 32301   │
│  42.59356851329265 │  -71.35754074987258 │ 01824   │
│          ·         │           ·         │   ·     │
│          ·         │           ·         │   ·     │
│          ·         │           ·         │   ·     │
│  42.86669753291775 │  -71.51274996857242 │ 03054   │
│  42.3646

In [ ]:
# set schema
# templated sql - DRY
schema = 'snow'

In [ ]:
# create compustat dimension
# MD5 hash as keys vs ordered int
# surrogate vs. natural keys
# primary keys (collisions)
sql = f"""


CREATE OR REPLACE TABLE {schema}.compustat (
    cusip_id VARCHAR,
    cusip_name VARCHAR,
    cusip_historical_id VARCHAR,
    cusip_historical_name VARCHAR,
    PRIMARY KEY (cusip_id)
);

"""

sql += f"""
INSERT INTO {schema}.compustat (cusip_id, cusip_name, cusip_historical_id, cusip_historical_name)
WITH cid AS (
    SELECT DISTINCT
        CUSIPHEADERID AS cusip_id,
        CUSIPHEADERNAME AS cusip_name,
        CUSIPHISTORYID AS cusip_historical_id,
        CUSIPHISTORYNAME AS cusip_historical_name
    FROM
        raw_companies
)
SELECT
    MD5(COALESCE(cusip_id || cusip_name || cusip_historical_id || cusip_historical_name, '-1')) AS cusip_id, --1 added for nulls
    cusip_name,
    cusip_historical_id,
    cusip_historical_name
FROM
    cid;
"""

con.sql(sql)

In [ ]:
con.sql('select * from snow.compustat limit 5')

┌──────────────────────────────────┬─────────────────────────────┬─────────────────────┬─────────────────────────────┐
│             cusip_id             │         cusip_name          │ cusip_historical_id │    cusip_historical_name    │
│             varchar              │           varchar           │       varchar       │           varchar           │
├──────────────────────────────────┼─────────────────────────────┼─────────────────────┼─────────────────────────────┤
│ 6bb61e3b7bce0931da574d19d1d82c88 │ NULL                        │ NULL                │ NULL                        │
│ fa7d01896b51cb06233e3ecb55b3e689 │ ALLIANT COMPUTER SYSTEMS CP │ 1880010             │ ALLIANT COMPUTER SYSTEMS CP │
│ 59036ad3d237e7c568673bee384b3ffd │ ENTERPRISE INFORMATICS INC  │ 2079910             │ ALPHAREL INC                │
│ 94af45e233e4d5f91cc562c17ef0f2e6 │ APOLLO COMPUTER INC         │ 3790410             │ APOLLO COMPUTER INC         │
│ 4ca2103c834adedbed35d0c7401b40ff │ AUDRE RECOG

In [ ]:
# create ceo dimension
# naming conventions
# joining on strings vs. others
sql = f"""

CREATE OR REPLACE TABLE {schema}.ceos (
    ceo_id VARCHAR,
    ceo_name VARCHAR,
    PRIMARY KEY (ceo_id)
);

"""

sql += f"""

INSERT INTO {schema}.ceos (ceo_id, ceo_name)

SELECT DISTINCT
    MD5(COALESCE(ceo, '-1')) AS ceo_id,
    ceo AS ceo_name
FROM
    raw_companies;

"""

con.sql(sql)

In [ ]:
con.table('snow.ceos')

┌──────────────────────────────────┬────────────────────────┐
│              ceo_id              │        ceo_name        │
│             varchar              │        varchar         │
├──────────────────────────────────┼────────────────────────┤
│ c35bc6945e2054750eb18ed6b9fed71a │ Kelly T. Hickel        │
│ dfa8c4619d6743917fbd6ce2be41ac1d │ Richard N. Spann       │
│ 41ec3b47cbc8d9e7af060eaa85864f75 │ James D. Norrod        │
│ f058b6eb7f7202e7fa1ee269a287a67c │ William T. Mason       │
│ 5fade36e274e7d6f120eb694eb658f81 │ Mark McMillen          │
│ e40fabee5589c9be667f61dd1d299af4 │ Dick Wauk              │
│ f30900f5f0907fcb2a46bc4e5a267fda │ Thomas J. Goliash      │
│ 2a62f8edc45ac28ffb0850a3e995353b │ Dominique F. Claessens │
│ 04244f82d9e865b19061a47f35cc5ad7 │ Ronald Gruner          │
│ d259b6064c2b1f1dfe67ea7449606e14 │ Craig J. Mundie        │
│                ·                 │        ·               │
│                ·                 │        ·               │
│       

In [ ]:
# create locations dimension
# type 2 dimensions - change slow
# insert only
# primary keys
sql = f"""

CREATE OR REPLACE TABLE {schema}.locations (
    location_id VARCHAR,
    zipcode VARCHAR,
    latitude DOUBLE,
    longitude DOUBLE,
    PRIMARY KEY (location_id)
);

"""

sql += f"""
INSERT INTO {schema}.locations (location_id, zipcode, latitude, longitude)

SELECT DISTINCT
    MD5(COALESCE(zip || CAST(lat AS VARCHAR) || CAST(lon AS VARCHAR), '-1')),
    zip,
    lat,
    lon
FROM
    raw_companies;

"""

con.sql(sql)

In [ ]:
# create companies dimension
# normalization benefits

sql = f"""

CREATE OR REPLACE TABLE {schema}.companies (
    company_id VARCHAR,
    company_name VARCHAR,
    cusip_id VARCHAR,
    ceo_id VARCHAR,
    founding_year INTEGER,
    ownership_type VARCHAR,
    exit_type VARCHAR,
    exit_comment VARCHAR,
    location_id VARCHAR,
    PRIMARY KEY (company_id)

);

"""

con.sql(sql)

sql += f"""

INSERT INTO {schema}.companies (company_id, company_name, cusip_id, ceo_id, founding_year, ownership_type, exit_type, exit_comment, location_id)

SELECT DISTINCT
    MD5(COALESCE(   COALESCE(firmname, '-1') ||
                    COALESCE(CAST(CUSIPHEADERID AS VARCHAR),'-1')||
                    COALESCE(CAST(firmid AS VARCHAR),'-1')||
                    COALESCE(CAST(found AS VARCHAR),'-1') ||
                    COALESCE(CAST(own AS VARCHAR),'-1') ||
                    COALESCE(CAST(fate AS VARCHAR),'-1') ||
                    COALESCE(CAST(estate AS VARCHAR),'-1') ||
                    COALESCE(ceo,'-1') ||
                    COALESCE(zip,'-1')
                    , '-1')) as id,
    firmname,
    MD5(COALESCE(CUSIPHEADERID || CUSIPHEADERNAME || CUSIPHISTORYID || CUSIPHISTORYNAME, '-1')),
    MD5(COALESCE(ceo, '-1')),
    found,
    CASE own
        WHEN 0 THEN 'Private'
        WHEN 1 THEN 'Public'
        WHEN 2 THEN 'Subsidiary'
        ELSE NULL END,
    CASE estate
        WHEN 0 THEN 'Censored'
        WHEN 1 THEN 'Exited market'
        WHEN 2 THEN 'Acquired'
        WHEN 3 THEN 'Spun off'
        WHEN 4 THEN 'Changed Name'
        ELSE NULL END,
    fate,
     MD5(COALESCE(zip || CAST(lat AS VARCHAR) || CAST(lon AS VARCHAR), '-1'))

FROM
    raw_companies

    ;
"""

con.sql(sql)

In [ ]:
# create product_offerings dimension
# what is a fact vs. a dimension
# CTE
sql = f"""

CREATE OR REPLACE TABLE {schema}.product_offerings (
    product_offering_id VARCHAR,
    product_types_offered VARCHAR,
    PRIMARY KEY (product_offering_id)
);

"""

sql += f"""

INSERT INTO {schema}.product_offerings (product_offering_id, product_types_offered)

WITH po AS (
     SELECT
        CASE
            WHEN system = 1 AND graphic = 0 THEN 'Desktop systems'
            WHEN system = 0 AND graphic = 1 THEN 'Graphics systems'
            WHEN system = 1 AND graphic = 1 THEN 'Desktop + Graphics systems'
            ELSE NULL END as offering

    from raw_companies
)
SELECT DISTINCT
    MD5(COALESCE(offering, '-1')),
    offering
FROM
    po;
"""

con.sql(sql)

In [ ]:
# create integration dimension
# type 4 dimension - change fast
sql = f"""

CREATE OR REPLACE TABLE {schema}.integrations (
   integration_id VARCHAR,
    cpu_source VARCHAR,
    os_source VARCHAR,
    application_source VARCHAR,
    communications_hardware_source VARCHAR,
    disk_source VARCHAR,
    ram_source VARCHAR,
    motherboard_source VARCHAR,
    PRIMARY KEY (integration_id)
);

"""

sql += f"""
INSERT INTO {schema}.integrations
(
    integration_id,
    cpu_source,
    os_source,
    application_source,
    communications_hardware_source,
    disk_source,
    ram_source,
    motherboard_source
)

SELECT DISTINCT
    MD5(
        CASE WHEN cpu = 0 THEN 'Bought' ELSE 'Produced' END ||
        CASE WHEN os = 0 THEN 'Bought' ELSE 'Produced' END ||
        CASE WHEN apps = 0 THEN 'Bought' ELSE 'Produced' END ||
        CASE WHEN comm = 0 THEN 'Bought' ELSE 'Produced' END ||
        CASE WHEN disk = 0 THEN 'Bought' ELSE 'Produced' END ||
        CASE WHEN memory = 0 THEN 'Bought' ELSE 'Produced' END ||
        CASE WHEN board = 0 THEN 'Bought' ELSE 'Produced' END
    ),
    CASE WHEN cpu = 0 THEN 'Bought' ELSE 'Produced' END,
    CASE WHEN os = 0 THEN 'Bought' ELSE 'Produced' END,
    CASE WHEN apps = 0 THEN 'Bought' ELSE 'Produced' END,
    CASE WHEN comm = 0 THEN 'Bought' ELSE 'Produced' END,
    CASE WHEN disk = 0 THEN 'Bought' ELSE 'Produced' END,
    CASE WHEN memory = 0 THEN 'Bought' ELSE 'Produced' END,
    CASE WHEN board = 0 THEN 'Bought' ELSE 'Produced' END
FROM
    raw_companies;


"""

con.sql(sql)

In [ ]:
### create years table by dynamically assigning values ###

# integer values as key which helps with partitioning.
# date table's grain goes, other date information
# CTAS

sql = """
CREATE OR REPLACE TABLE years AS

WITH RECURSIVE years AS (
    SELECT 1970 as year
    UNION ALL
    SELECT year + 1
    FROM years
    WHERE year < 2000
)
SELECT
    year,
    CASE
        WHEN MOD(year - 1970, 12) = 0 THEN 'Dog'
        WHEN MOD(year - 1970, 12) = 1 THEN 'Pig'
        WHEN MOD(year - 1970, 12) = 2 THEN 'Rat'
        WHEN MOD(year - 1970, 12) = 3 THEN 'Ox'
        WHEN MOD(year - 1970, 12) = 4 THEN 'Tiger'
        WHEN MOD(year - 1970, 12) = 5 THEN 'Rabbit'
        WHEN MOD(year - 1970, 12) = 6 THEN 'Dragon'
        WHEN MOD(year - 1970, 12) = 7 THEN 'Snake'
        WHEN MOD(year - 1970, 12) = 8 THEN 'Horse'
        WHEN MOD(year - 1970, 12) = 9 THEN 'Sheep'
        WHEN MOD(year - 1970, 12) = 10 THEN 'Monkey'
        WHEN MOD(year - 1970, 12) = 11 THEN 'Rooster'
    END as zodiac_type
FROM years;
"""

con.sql(sql)

### making the fact table

In [ ]:
# create workstation_sales table
# additive facts vs. non-additive

sql = f"""

CREATE OR REPLACE TABLE {schema}.workstation_sales (
   year_id INTEGER,
    company_id VARCHAR,
    integration_id VARCHAR,
    product_offering_id VARCHAR,
    sales DOUBLE,
    research_budget DOUBLE,
    employee_count DOUBLE,
    product_offering_count INTEGER,
    product_category_count INTEGER,
);

"""

con.sql(sql)

sql += f"""

INSERT INTO {schema}.workstation_sales
   (year_id,
    company_id ,
    integration_id ,
    product_offering_id ,
    sales ,
    research_budget ,
    employee_count ,
    product_offering_count ,
    product_category_count
    )

SELECT
    year,
    MD5(COALESCE(   COALESCE(firmname, '-1') ||
                    COALESCE(CAST(CUSIPHEADERID AS VARCHAR),'-1')||
                    COALESCE(CAST(firmid AS VARCHAR),'-1')||
                    COALESCE(CAST(found AS VARCHAR),'-1') ||
                    COALESCE(CAST(own AS VARCHAR),'-1') ||
                    COALESCE(CAST(fate AS VARCHAR),'-1') ||
                    COALESCE(CAST(estate AS VARCHAR),'-1') ||
                    COALESCE(ceo,'-1') ||
                    COALESCE(zip,'-1')
                    , '-1')) as id,

    MD5(
        CASE WHEN cpu = 0 THEN 'Bought' ELSE 'Produced' END ||
        CASE WHEN os = 0 THEN 'Bought' ELSE 'Produced' END ||
        CASE WHEN apps = 0 THEN 'Bought' ELSE 'Produced' END ||
        CASE WHEN comm = 0 THEN 'Bought' ELSE 'Produced' END ||
        CASE WHEN disk = 0 THEN 'Bought' ELSE 'Produced' END ||
        CASE WHEN memory = 0 THEN 'Bought' ELSE 'Produced' END ||
        CASE WHEN board = 0 THEN 'Bought' ELSE 'Produced' END
    ),

    MD5(COALESCE(
        CASE
            WHEN system = 1 AND graphic = 0 THEN 'Desktop systems'
            WHEN system = 0 AND graphic = 1 THEN 'Graphics systems'
            WHEN system = 1 AND graphic = 1 THEN 'Desktop + Graphics systems'
            ELSE NULL END,
        '-1'
    )),

    sales,
    randd,
    employ,
    products,
    ptypes


FROM
    raw_companies
    ;

"""

con.sql(sql)

In [ ]:
# con.sql('select count(*) from workstation_snowflake.workstation_sales a join workstation_snowflake.companies b on a.company_id = b.company_id')

# con.sql('select count(*) from workstation_snowflake.workstation_sales a join workstation_snowflake.integrations b on a.integration_id = b.integration_id')

# con.sql('select count(*) from workstation_snowflake.companies') # 388

# con.sql("select 'locations', count(*) from workstation_snowflake.companies c join workstation_snowflake.locations l on c.location_id = l.location_id")

con.sql("select 'locations', count(*) from snow.companies c join snow.compustat s on c.cusip_id = s.cusip_id")

# con.sql("select 'locations', count(*) from workstation_snowflake.companies c join workstation_snowflake.ceos s on c.ceo_id = s.ceo_id")

# con.sql('select company_name, c.location_id, l.location_id from workstation_snowflake.companies c left join workstation_snowflake.locations l on c.location_id = l.location_id')

# con.sql('select count(*) from raw_companies')

┌─────────────┬──────────────┐
│ 'locations' │ count_star() │
│   varchar   │    int64     │
├─────────────┼──────────────┤
│ locations   │          388 │
└─────────────┴──────────────┘

## A star data model

Steps of Kimball dimensional modeling:

1. Select the business process
2. Declare the grain
3. Identify the dimensions
4. Identify the facts

![](star.png)

In [ ]:
# update schema
schema = 'star'

In [ ]:
# make new companies attribute which contains
# pros/cons of denormalizing to star
sql = f"""

CREATE OR REPLACE TABLE {schema}.companies (
       company_id VARCHAR,
    company_name VARCHAR,
    cusip_id VARCHAR,
    cusip_name VARCHAR,
    cusip_historical_id VARCHAR,
    cusip_historical_name VARCHAR,
    ceo_name VARCHAR,
    founding_year INTEGER,
    ownership_type VARCHAR,
    exit_type VARCHAR,
    exit_comment VARCHAR,
    zipcode VARCHAR,
    latitude DOUBLE,
    longitude DOUBLE,
    PRIMARY KEY (company_id),
);

"""

con.sql(sql)

sql += f"""

INSERT INTO {schema}.companies (
    company_id ,
    company_name ,
    cusip_id ,
    cusip_name ,
    cusip_historical_id ,
    cusip_historical_name ,
    ceo_name ,
    founding_year ,
    ownership_type ,
    exit_type ,
    exit_comment ,
    zipcode ,
    latitude ,
    longitude )

SELECT DISTINCT
    MD5(COALESCE(   COALESCE(firmname, '-1') ||
                    COALESCE(CAST(CUSIPHEADERID AS VARCHAR),'-1')||
                    COALESCE(CAST(firmid AS VARCHAR),'-1')||
                    COALESCE(CAST(found AS VARCHAR),'-1') ||
                    COALESCE(CAST(own AS VARCHAR),'-1') ||
                    COALESCE(CAST(fate AS VARCHAR),'-1') ||
                    COALESCE(CAST(estate AS VARCHAR),'-1') ||
                    COALESCE(ceo,'-1') ||
                    COALESCE(zip,'-1')
                    , '-1')) as id,
    firmname,
    CUSIPHEADERID,
    CUSIPHeaderName,
    CUSIPHISTORYID,
    CUSIPHistoryName,
    ceo,
    found,
    CASE own
        WHEN 0 THEN 'Private'
        WHEN 1 THEN 'Public'
        WHEN 2 THEN 'Subsidiary'
        ELSE NULL END,
    CASE estate
        WHEN 0 THEN 'Censored'
        WHEN 1 THEN 'Exited market'
        WHEN 2 THEN 'Acquired'
        WHEN 3 THEN 'Spun off'
        WHEN 4 THEN 'Changed Name'
        ELSE NULL END,
    fate,
    ZIP,
    lat,
    lon

FROM
    raw_companies;
"""

con.sql(sql)

In [ ]:
con.sql('select count(*) from workstation_snowflake.workstation_sales a join star.companies b on a.company_id = b.company_id')

### Some query testing

In [ ]:
con.sql("""
select zipcode, sum(sales)
from snow.workstation_sales a
join snow.companies c on a.company_id=c.company_id
join snow.locations l on c.location_id=l.location_id
group by 1 order by 2 desc""")

┌─────────┬────────────────┐
│ zipcode │   sum(sales)   │
│ varchar │     double     │
├─────────┼────────────────┤
│ 10504   │ 598003998720.0 │
│ 45479   │ 154701496704.0 │
│ 94304   │ 141994998272.0 │
│ 01754   │ 132260221440.0 │
│ 95054   │ 121301996099.0 │
│ 14650   │  95705997312.0 │
│ 01821   │  81184972152.0 │
│ 95134   │  67506182944.0 │
│ 07660   │  48000000000.0 │
│ 95008   │  36615437056.0 │
│   ·     │             ·  │
│   ·     │             ·  │
│   ·     │             ·  │
│ 15276   │            1.0 │
│ 97006   │            1.0 │
│ 41048   │            1.0 │
│ 27709   │           NULL │
│ 92803   │           NULL │
│ 97124   │           NULL │
│ 94063   │           NULL │
│ 94577   │           NULL │
│ 19380   │           NULL │
│ 77094   │           NULL │
├─────────┴────────────────┤
│   156 rows (20 shown)    │
└──────────────────────────┘

In [ ]:
con.sql("""
select zipcode, sum(sales)
from snow.workstation_sales a
join star.companies c on a.company_id=c.company_id
group by 1 order by 2 desc""")

┌─────────┬────────────────┐
│ zipcode │   sum(sales)   │
│ varchar │     double     │
├─────────┼────────────────┤
│ 10504   │ 598003998720.0 │
│ 45479   │ 154701496704.0 │
│ 94304   │ 141994998272.0 │
│ 01754   │ 132260221440.0 │
│ 95054   │ 121301996099.0 │
│ 14650   │  95705997312.0 │
│ 01821   │  81184972152.0 │
│ 95134   │  67506182944.0 │
│ 07660   │  48000000000.0 │
│ 95008   │  36615437056.0 │
│   ·     │             ·  │
│   ·     │             ·  │
│   ·     │             ·  │
│ 15276   │            1.0 │
│ 97006   │            1.0 │
│ 41048   │            1.0 │
│ 27709   │           NULL │
│ 92803   │           NULL │
│ 97124   │           NULL │
│ 94063   │           NULL │
│ 94577   │           NULL │
│ 19380   │           NULL │
│ 77094   │           NULL │
├─────────┴────────────────┤
│   156 rows (20 shown)    │
└──────────────────────────┘

In [ ]:
%%timeit -n 10

con.sql("""
select zipcode, sum(sales)
from snow.workstation_sales a
join snow.companies c on a.company_id=c.company_id
join snow.locations l on c.location_id=l.location_id
group by 1 order by 2 desc""")

117 µs ± 13.3 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%%timeit -n 10

con.sql("""
select zipcode, sum(sales)
from snow.workstation_sales a
join star.companies c on a.company_id=c.company_id
group by 1 order by 1 desc""")

109 µs ± 13 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
